# Dev Tools

Let's start with the simplest CUDA kernel.  Observe the error messages that occur when executing the following cell.  Notice that we are building the executable and then running it with `compute-sanitizer`.

In [21]:
import os

if os.getenv("COLAB_RELEASE_TAG"): # If running in Google Colab:
  !mkdir -p Sources
  !wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.02-Kernels/Sources/ach.h -nv -O Sources/ach.h

2025-12-26 08:12:09 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.02-Kernels/Sources/ach.h [4416/4416] -> "Sources/ach.h" [1]


In [22]:
!nvcc --extended-lambda -g -G -o /tmp/a.out Solutions/row-symmetry-check.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable
!compute-sanitizer /tmp/a.out # run sanitizer

/usr/bin/ld: /tmp/tmpxft_0000195e_00000000-12_row-symmetry-check.o: in function `main':
/content/Solutions/ach.cuh:132: undefined reference to `simulate(cuda::std::__4::mdspan<float, cuda::std::__4::extents<int, 18446744073709551615ul, 18446744073709551615ul>, cuda::std::__4::layout_right, cuda::std::__4::default_accessor<float> >, float*, CUstream_st*)'
collect2: error: ld returned 1 exit status
/bin/bash: line 1: /tmp/a.out: No such file or directory
========= COMPUTE-SANITIZER
========= Target application doesn't exist or is not a valid executable


There are a lot of error messages printed, and the instructive errors are at the very top.  Note one of the errors printed `Invalid __global__ read of size 4 bytes at symmetry_check_kernel`.  This tells you exactly where to look to find the memory access error.


```
Invalid __global__ read of size 4 bytes
=========     at symmetry_check_kernel(cuda::std::__4::mdspan<float, cuda::std::__4::extents<int, (unsigned long)18446744073709551615, (unsigned long)18446744073709551615>, cuda::std::__4::layout_right, cuda::std::__4::default_accessor<float>>, int)+0x2490 in 03.02-Kernels/Solutions/row-symmetry-check.cu:6
=========     by thread (928,0,0) in block (4,0,0)
=========     Address 0x709388060 is out of bounds
=========     and is 97 bytes after the nearest allocation at 0x708000000 of size 20480000 bytes
```

The code below fixes the error.  Note the use of the `if (column < temp.extent(1))` statement, which guards the execution of the thread.  Each thread checks whether its `column` is less than the size of the array, `temp`.  If it is less, it executes the symmetry check, but if it is NOT, then it just returns.  This type of simple fix is very common in CUDA kernel programming to ensure that threads don't access out-of-bounds memory.

Execute the next two cells and verify that `compute-sanitizer` does not report any further errors.

In [23]:
%%writefile Sources/row-symmetry-check-fixed.cpp
#include "ach.h"

__global__ void symmetry_check_kernel(ach::temperature_grid_f temp, int row)
{
  int column = blockIdx.x * blockDim.x + threadIdx.x;

  if (column < temp.extent(1))
  {
    if (abs(temp(row, column) - temp(temp.extent(0) - 1 - row, column)) > 0.1)
    {
        printf("Error: asymmetry in %d / %d\n", column, temp.extent(1));
    }
  }
}

void symmetry_check(ach::temperature_grid_f temp_in, cudaStream_t stream)
{
  int width      = temp_in.extent(1);
  int block_size = 1024;
  int grid_size  = cuda::ceil_div(width, block_size);

  int target_row = 0;
  symmetry_check_kernel<<<grid_size, block_size, 0, stream>>>(temp_in, target_row);
}

void simulate(ach::temperature_grid_f temp_in, float *temp_out, cudaStream_t stream)
{
  symmetry_check(temp_in, stream);
  ach::simulate(temp_in, temp_out, stream);
}

Writing Sources/row-symmetry-check-fixed.cpp


In [24]:
!nvcc --extended-lambda -o /tmp/a.out Sources/row-symmetry-check-fixed.cpp -x cu -arch=native # build executable
!compute-sanitizer /tmp/a.out # run sanitizer

========= COMPUTE-SANITIZER
compute in 0.426901 s
========= ERROR SUMMARY: 0 errors


---
You've learned a lot about CUDA kernels!  Jump to the [next section](../03.03-Atomics/03.03.01-Histogram.ipynb).